In [ ]:
# Cell 1 : Standard Library Imports

import os
import time
import logging
import joblib
import warnings
import random
from datetime import datetime, timedelta

# Third-Party Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import xgboost as xgb
import yfinance as yf
import keras_tuner as kt
import matplotlib as mpl
import matplotlib.dates as mdates
import ta
import tensorflow as tf
from sklearn.linear_model import RidgeCV
import pandas_market_calendars as mcal
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, GRU, Bidirectional, Input, Dropout, Dense
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler, StandardScaler, MinMaxScaler
from sklearn.metrics import (mean_squared_error, mean_absolute_error, 
                             mean_absolute_percentage_error, r2_score)
from sklearn.ensemble import RandomForestRegressor
from scipy.stats import skew, kurtosis, shapiro
from statsmodels.stats.outliers_influence import variance_inflation_factor
from tensorflow.keras.layers import Input, Dense, LayerNormalization, Dropout, MultiHeadAttention, Embedding
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers.schedules import LearningRateSchedule
from xgboost import XGBRegressor

In [ ]:
# Cell 2: Fetch the Stock Data (Time-series Only)

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Initialize dictionaries to store dataframes
daily_data_dict = {}

# List of stocks to fetch data for
stocks = ['AAPL', 'MSFT', 'GOOGL', 'AMZN']

# Define the time frames for data
end_date = datetime.now()
start_date_daily = end_date - timedelta(days=10*365)   # 10 years of daily data

# Create directories for the data
os.makedirs('../data/stock_data', exist_ok=True)

# Function to fetch stock data
def fetch_stock_data(ticker, start, end, interval):
    try:
        data = yf.download(ticker, start=start, end=end, interval=interval)
        if data.empty:
            logging.warning(f"No data retrieved for {ticker} from {start} to {end} with interval {interval}")
        return data.drop(columns=['Adj Close'], errors='ignore')
    except Exception as e:
        logging.error(f"Error fetching data for {ticker}: {e}")
        return pd.DataFrame()

# Fetch and save daily time-series data
for stock in tqdm(stocks, desc="Fetching stocks data"):

    # Daily Data (10 years)
    daily_data = fetch_stock_data(stock, start_date_daily, end_date, '1d')
    if not daily_data.empty:
        daily_data_dict[stock] = daily_data
        daily_data.to_csv(f'../data/stock_data/{stock}_daily.csv', index=True)

    # Add a delay to avoid API rate limits
    time.sleep(2)

print("Time-series data fetching and saving complete.")

In [ ]:
# Cell 3A: Feature Engineering for Daily Data - Basic Technical Indicators

# Define directories (assuming already defined)
raw_data_dir = '../data/stock_data'

# Initialize storage for enriched daily data
enriched_daily_data_dict = {}

# Feature Engineering Functions
def add_basic_technical_indicators_daily(df):
    # Simple and Exponential Moving Averages
    df['SMA_20'] = ta.trend.SMAIndicator(close=df['Close'], window=20, fillna=False).sma_indicator()
    df['EMA_20'] = ta.trend.EMAIndicator(close=df['Close'], window=20, fillna=False).ema_indicator()

    # Relative Strength Index
    df['RSI_14'] = ta.momentum.RSIIndicator(close=df['Close'], window=14, fillna=False).rsi()

    # Moving Average Convergence Divergence
    macd = ta.trend.MACD(close=df['Close'], fillna=False)
    df['MACD'] = macd.macd()
    df['MACD_signal'] = macd.macd_signal()
    df['MACD_diff'] = macd.macd_diff()

    # Average True Range
    df['ATR_14'] = ta.volatility.AverageTrueRange(
        high=df['High'], low=df['Low'], close=df['Close'], window=14, fillna=False
    ).average_true_range()

    # Bollinger Bands
    bollinger = ta.volatility.BollingerBands(close=df['Close'], window=20, window_dev=2, fillna=False)
    df['Bollinger_High'] = bollinger.bollinger_hband()
    df['Bollinger_Low'] = bollinger.bollinger_lband()
    df['Bollinger_Width'] = bollinger.bollinger_wband()
    
    # On-Balance Volume
    df['OBV'] = ta.volume.OnBalanceVolumeIndicator(close=df['Close'], volume=df['Volume'], fillna=False).on_balance_volume()
    
    # Stochastic Oscillator
    stoch = ta.momentum.StochasticOscillator(
        high=df['High'], low=df['Low'], close=df['Close'], window=14, smooth_window=3, fillna=False
    )
    df['Stoch_%K'] = stoch.stoch()
    df['Stoch_%D'] = stoch.stoch_signal()

    # Rate of Change
    df['ROC_10'] = ta.momentum.ROCIndicator(close=df['Close'], window=10, fillna=False).roc()
    
    # Daily Returns
    df['Daily_Return'] = df['Close'].pct_change()
    
    # Close-Open Percent
    df['Close_Open_Percent'] = (df['Close'] - df['Open']) / df['Open'] * 100
    
    # High-Low Percent
    df['High_Low_Percent'] = (df['High'] - df['Low']) / df['Low'] * 100

    return df

def process_daily_stock_data_basic(stock):
    daily_file = os.path.join(raw_data_dir, f"{stock}_daily.csv")
    if not os.path.exists(daily_file):
        logging.warning(f"Daily data file not found for {stock}: {daily_file}")
        return

    daily_df = pd.read_csv(daily_file, parse_dates=['Date'])
    required_columns = ['Open', 'High', 'Low', 'Close', 'Volume']
    if not all(col in daily_df.columns for col in required_columns):
        logging.error(f"Missing required columns in {stock}'s data. Skipping.")
        return

    # Ensure numeric data and handle missing values
    daily_df[required_columns] = daily_df[required_columns].apply(pd.to_numeric, errors='coerce')
    daily_df.sort_values('Date', inplace=True)
    daily_df.drop_duplicates(subset=['Date'], inplace=True)
    daily_df.dropna(subset=required_columns, inplace=True)

    # Apply Basic Technical Indicators
    daily_df = add_basic_technical_indicators_daily(daily_df)

    # Drop rows with NaN in critical features
    critical_features = ['SMA_20', 'EMA_20', 'RSI_14', 'MACD', 'MACD_signal', 'MACD_diff', 'ATR_14',
                         'Bollinger_High', 'Bollinger_Low', 'Bollinger_Width', 'OBV', 'Stoch_%K', 'Stoch_%D',
                         'ROC_10', 'Daily_Return', 'Close_Open_Percent', 'High_Low_Percent']
    daily_df.dropna(subset=critical_features, inplace=True)
    
    # Store the enriched DataFrame
    enriched_daily_data_dict[stock] = daily_df
    
    # Logging
    logging.info(f"Basic technical indicators added for daily data of {stock}")
    logging.info(f"Sample features:\n{daily_df.head()}")

def verify_feature_engineered_data_basic(stock):
    df = enriched_daily_data_dict.get(stock)
    if df is None:
        print(f"No data found for {stock}.")
        return
    
    print(f"Verifying data for {stock}...")
    
    nan_counts = df.isna().sum()
    missing = nan_counts[nan_counts > 0]
    if not missing.empty:
        print(f"NaN counts:\n{missing}")
        print("WARNING: There are still NaN values in the features.")
    else:
        print("All features are free of NaN values.")
    print("-" * 80)

def feature_engineer_daily_stock_data_basic(stock):
    process_daily_stock_data_basic(stock)
    verify_feature_engineered_data_basic(stock)

# Process each stock's daily data
for stock in daily_data_dict.keys():
    feature_engineer_daily_stock_data_basic(stock)
    
print("Basic technical indicators added for all daily stock data.")

In [ ]:
# Cell 3B: Feature Engineering for Daily Data - Advanced Features

def calculate_vwap(df):
    """Calculate Volume Weighted Average Price (VWAP)."""
    typical_price = (df['High'] + df['Low'] + df['Close']) / 3
    vwap = (typical_price * df['Volume']).cumsum() / df['Volume'].cumsum()
    return vwap

def add_advanced_features_daily(df):
    # a. Relative Strength Index (RSI_14)
    rsi = ta.momentum.RSIIndicator(close=df['Close'], window=14, fillna=False)
    df['RSI_14'] = rsi.rsi()

    # b. Moving Average Convergence Divergence (MACD)
    macd = ta.trend.MACD(close=df['Close'], window_slow=26, window_fast=12, window_sign=9, fillna=False)
    df['MACD'] = macd.macd()
    df['MACD_diff'] = macd.macd_diff()

    # c. Average True Range (ATR_14)
    atr = ta.volatility.AverageTrueRange(high=df['High'], low=df['Low'], close=df['Close'], window=14, fillna=False)
    df['ATR_14'] = atr.average_true_range()

    # d. Bollinger Bands Width (Bollinger_Width)
    bollinger = ta.volatility.BollingerBands(close=df['Close'], window=20, window_dev=2, fillna=False)
    df['Bollinger_High'] = bollinger.bollinger_hband()
    df['Bollinger_Low'] = bollinger.bollinger_lband()
    df['Bollinger_Width'] = bollinger.bollinger_wband()

    # e. Keltner Channels
    keltner = ta.volatility.KeltnerChannel(
        high=df['High'], low=df['Low'], close=df['Close'], window=20, window_atr=10, fillna=False
    )
    df['Keltner_High'] = keltner.keltner_channel_hband()
    df['Keltner_Low'] = keltner.keltner_channel_lband()
    df['Keltner_Width'] = df['Keltner_High'] - df['Keltner_Low']

    # f. Chaikin Money Flow (CMF)
    cmf_indicator = ta.volume.ChaikinMoneyFlowIndicator(
        high=df['High'], low=df['Low'], close=df['Close'], volume=df['Volume'], window=20, fillna=False
    )
    df['CMF'] = cmf_indicator.chaikin_money_flow()

    # 2. Lag Features
    for lag in range(1, 4):
        df[f'Lag_Close_{lag}'] = df['Close'].shift(lag)

    # 3. Moving Averages and Standard Deviations
    df['MA_5'] = df['Close'].rolling(window=5, min_periods=5).mean()
    df['MA_Volume_10'] = df['Volume'].rolling(window=10, min_periods=10).mean()
    df['STD_Close_5'] = df['Close'].rolling(window=5, min_periods=5).std()
    df['MA_RSI_10'] = df['RSI_14'].rolling(window=10, min_periods=10).mean()
    df['MA_MACD_10'] = df['MACD'].rolling(window=10, min_periods=10).mean()
    df['MA_ATR_10'] = df['ATR_14'].rolling(window=10, min_periods=10).mean()
    df['MA_Bollinger_Width_10'] = df['Bollinger_Width'].rolling(window=10, min_periods=10).mean()

    # 4. VWAP and its Moving Average
    df['VWAP'] = calculate_vwap(df)
    df['MA_VWAP_10'] = df['VWAP'].rolling(window=10, min_periods=10).mean()

    # 5. Date Features
    if 'Date' not in df.columns:
        raise KeyError("Missing 'Date' column required for date-based feature engineering.")
    df['Day_of_Week'] = df['Date'].dt.dayofweek
    df['Month'] = df['Date'].dt.month

    # 6. Rolling Correlations and Momentum
    df['Rolling_Corr_Close_Volume_10'] = df['Close'].rolling(window=10, min_periods=10).corr(df['Volume'])
    df['Momentum_5'] = df['Close'].diff(5)
    df['Volatility_10'] = df['Close'].rolling(window=10, min_periods=10).std()

    # 7. Cyclical Encoding for Day of Week and Month
    df['Day_of_Week_sin'] = np.sin(2 * np.pi * df['Day_of_Week'] / 7)
    df['Day_of_Week_cos'] = np.cos(2 * np.pi * df['Day_of_Week'] / 7)
    df['Month_sin'] = np.sin(2 * np.pi * df['Month'] / 12)
    df['Month_cos'] = np.cos(2 * np.pi * df['Month'] / 12)

    # Drop original Day_of_Week and Month
    df.drop(['Day_of_Week', 'Month'], axis=1, inplace=True)

    # 8. Interaction Features
    df['MACD_diff_RSI_14'] = df['MACD_diff'] * df['RSI_14']
    df['MACD_diff_ATR_14'] = df['MACD_diff'] * df['ATR_14']
    df['RSI_14_ATR_14'] = df['RSI_14'] * df['ATR_14']

    # 9. Trend and Seasonal Components
    df['Trend'] = df['Close'].rolling(window=30, min_periods=30).mean()
    df['Seasonal'] = df['Close'] - df['Trend']

    # 10. Rolling Skewness and Kurtosis
    df['Rolling_Skew_Close_10'] = df['Close'].rolling(window=10, min_periods=10).skew()
    df['Rolling_Kurt_Close_10'] = df['Close'].rolling(window=10, min_periods=10).kurt()

    # Fill any remaining NaN values
    df.fillna(method='bfill', inplace=True)
    df.fillna(method='ffill', inplace=True)

    return df

def process_daily_stock_data_advanced(stock):
    """Process and add advanced features to a single stock's DataFrame."""
    df = enriched_daily_data_dict.get(stock)
    if df is None:
        logging.warning(f"No basic technical indicator data found for {stock}. Skipping advanced feature engineering.")
        return
    
    # Apply Advanced Feature Engineering
    df = add_advanced_features_daily(df)
    
    # Define critical advanced features to check for NaNs
    advanced_features = [
        'Lag_Close_1', 'Lag_Close_2', 'Lag_Close_3',
        'MA_5', 'MA_Volume_10', 'STD_Close_5',
        'MA_RSI_10', 'MA_MACD_10', 'MA_ATR_10', 'MA_Bollinger_Width_10',
        'VWAP', 'MA_VWAP_10',
        'Rolling_Corr_Close_Volume_10', 'Momentum_5', 'Volatility_10',
        'Day_of_Week_sin', 'Day_of_Week_cos', 'Month_sin', 'Month_cos',
        'MACD_diff_RSI_14', 'MACD_diff_ATR_14', 'RSI_14_ATR_14',
        'Trend', 'Seasonal',
        'Keltner_High', 'Keltner_Low', 'Keltner_Width',
        'CMF',
        'Rolling_Skew_Close_10', 'Rolling_Kurt_Close_10'
    ]
    
    # Drop rows with NaN in advanced features
    df.dropna(subset=advanced_features, inplace=True)
    
    # Store the updated DataFrame back into the dictionary
    enriched_daily_data_dict[stock] = df
    
    # Logging
    logging.info(f"Advanced features added for daily data of {stock}")
    logging.info(f"Sample advanced features for {stock}:\n{df.head()}")

def verify_feature_engineered_data_advanced(stock):
    """Verify that advanced features are free of NaN values."""
    df = enriched_daily_data_dict.get(stock)
    if df is None:
        print(f"No data found for {stock}.")
        return
    
    print(f"Verifying advanced features for {stock}...")
    
    nan_counts = df.isna().sum()
    missing = nan_counts[nan_counts > 0]
    if not missing.empty:
        print(f"NaN counts in advanced features:\n{missing}")
        print("WARNING: There are still NaN values in the advanced features.")
    else:
        print("All advanced features are free of NaN values.")
    print("-" * 80)

def feature_engineer_daily_stock_data_advanced(stock):
    """Wrapper function to process and verify advanced features for a stock."""
    process_daily_stock_data_advanced(stock)
    verify_feature_engineered_data_advanced(stock)

# Process each stock's daily data for advanced features
for stock in enriched_daily_data_dict.keys():
    feature_engineer_daily_stock_data_advanced(stock)
    
print("Advanced features added for all daily stock data.")


In [ ]:
# Cell 4: Splitting Daily Data into Training and Testing Sets

import logging
from sklearn.preprocessing import MinMaxScaler

# Function to split time series data
def split_time_series_data(df, date_column, target_column, split_ratio=0.8):
    # Sort the DataFrame by the date/time column
    df_sorted = df.sort_values(by=date_column).reset_index(drop=True)
    
    # Determine the split index
    split_index = int(len(df_sorted) * split_ratio)
    
    # Split the data
    train_df = df_sorted.iloc[:split_index]
    test_df = df_sorted.iloc[split_index:]
    
    # Separate features and target
    X_train = train_df.drop(columns=[target_column, date_column])
    y_train = train_df[target_column]
    
    X_test = test_df.drop(columns=[target_column, date_column])
    y_test = test_df[target_column]
    
    return X_train, X_test, y_train, y_test

# Function to split the data for each stock
def split_daily_data_dict(data_dict, date_column, target_column, split_ratio=0.8):
    split_dict = {}
    
    for stock, df in data_dict.items():
        logging.info(f"Splitting data for {stock}...")
        X_train, X_test, y_train, y_test = split_time_series_data(df, date_column, target_column, split_ratio)
        split_dict[stock] = {
            'X_train': X_train,
            'X_test': X_test,
            'y_train': y_train,
            'y_test': y_test
        }
        logging.info(f"Completed splitting for {stock}.")
    
    return split_dict

# Function to verify the split data
def verify_split(split_dict, stocks):
    for stock in stocks:
        data = split_dict.get(stock, {})
        if not data:
            print(f"No split data found for {stock}.")
            continue
        
        X_train = data.get('X_train')
        X_test = data.get('X_test')
        y_train = data.get('y_train')
        y_test = data.get('y_test')
        
        print(f"Verifying split for {stock}:")
        print(f" - Training set size: {X_train.shape[0]} samples")
        print(f" - Testing set size: {X_test.shape[0]} samples")
        print(f" - Feature columns in Training set: {X_train.columns.tolist()}")
        print(f" - Feature columns in Testing set: {X_test.columns.tolist()}")
        print("-" * 80)

# Define parameters
daily_date_column = 'Date'    # Updated to match your data
daily_target_column = 'Close' # Updated to match your data

# Split Daily Data
logging.info("Starting to split Daily Data...")
split_daily_data = split_daily_data_dict(
    enriched_daily_data_dict,
    date_column=daily_date_column,
    target_column=daily_target_column,
    split_ratio=0.8
)
logging.info("Completed splitting Daily Data.")

# Verification
print("\nVerifying Split for Daily Data:")
verify_split(split_daily_data, enriched_daily_data_dict.keys())

In [ ]:
# Cell 5: Scaling the Data Using MinMaxScaler (Including Targets)

# Function to scale data using MinMaxScaler for both features and targets
def scale_data_with_target(split_data_dict):
    scaled_data_dict = {}
    
    # Directories to save scalers
    scaler_save_dir = '../models/scalers'
    os.makedirs(scaler_save_dir, exist_ok=True)
    
    for stock, data in split_data_dict.items():
        logging.info(f"Scaling data for {stock}...")
        
        X_train = data['X_train']
        X_test = data['X_test']
        y_train = data['y_train'].values.reshape(-1, 1)  # Reshape for scaler
        y_test = data['y_test'].values.reshape(-1, 1)
        
        # Initialize scalers
        scaler_X = MinMaxScaler()
        scaler_y = MinMaxScaler()
        
        # Fit scalers on training data and transform both training and testing data
        X_train_scaled = pd.DataFrame(scaler_X.fit_transform(X_train), columns=X_train.columns, index=X_train.index)
        X_test_scaled = pd.DataFrame(scaler_X.transform(X_test), columns=X_test.columns, index=X_test.index)
        
        y_train_scaled = scaler_y.fit_transform(y_train).flatten()  # Flatten to 1D array
        y_test_scaled = scaler_y.transform(y_test).flatten()
        
        # Save scalers using joblib
        scaler_X_path = os.path.join(scaler_save_dir, f'minmax_scaler_X_{stock}.joblib')
        scaler_y_path = os.path.join(scaler_save_dir, f'minmax_scaler_y_{stock}.joblib')
        joblib.dump(scaler_X, scaler_X_path)
        joblib.dump(scaler_y, scaler_y_path)
        logging.info(f"Scalers saved for {stock} at {scaler_X_path} and {scaler_y_path}.")
        
        # Update the scaled data dictionary
        scaled_data_dict[stock] = {
            'X_train_scaled': X_train_scaled,
            'X_test_scaled': X_test_scaled,
            'y_train_scaled': y_train_scaled,
            'y_test_scaled': y_test_scaled,
            'scaler_X': scaler_X,
            'scaler_y': scaler_y
        }
        
        logging.info(f"Completed scaling for {stock}.")
    
    return scaled_data_dict

# Scale the split daily data with target
logging.info("Starting to scale Daily Data with target...")
scaled_daily_data = scale_data_with_target(split_daily_data)
logging.info("Completed scaling Daily Data with target.")

# Verification
print("\nVerifying Scaled Data for Daily Data (Including Targets):")
for stock in scaled_daily_data.keys():
    data = scaled_daily_data[stock]
    X_train_scaled = data['X_train_scaled']
    X_test_scaled = data['X_test_scaled']
    y_train_scaled = data['y_train_scaled']
    y_test_scaled = data['y_test_scaled']
    
    print(f"Scaled data for {stock}:")
    print(f" - Scaled Training set shape: {X_train_scaled.shape}, Scaled Training targets shape: {y_train_scaled.shape}")
    print(f" - Scaled Testing set shape: {X_test_scaled.shape}, Scaled Testing targets shape: {y_test_scaled.shape}")
    print(f" - Feature columns: {X_train_scaled.columns.tolist()}")
    print("-" * 80)


In [ ]:
# Cell 6: Training and Evaluating LSTM Models for Daily Data

# Set random seeds for reproducibility
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)
random.seed(SEED)

# Define Parameters
TIMESTEPS = 60  # Number of past days to use for prediction
BATCH_SIZE = 32
EPOCHS = 100  # Increased to allow more training
VALIDATION_SPLIT = 0.1  # Fraction of training data to use for validation

# Define Evaluation Metrics Function
def evaluate_model(y_true, y_pred):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    return rmse, mae, r2

# Function to Create Sequences
def create_sequences(X, y, timesteps):
    X_seq, y_seq = [], []
    for i in range(timesteps, len(X)):
        X_seq.append(X[i-timesteps:i].values)
        y_seq.append(y[i])
    return np.array(X_seq), np.array(y_seq)

# Function to Build LSTM Model
def build_lstm_model(input_shape):
    model = Sequential()
    model.add(Input(shape=input_shape)) # Explicit Input Layer
    model.add(LSTM(units=50, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(units=50, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(units=25, activation='relu'))
    model.add(Dense(units=1))  # Output layer
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# Initialize a dictionary to store model performance
model_performance = {}

# Directories to save models and scalers
model_save_dir = '../models/lstm_models'
scaler_save_dir = '../models/scalers'
os.makedirs(model_save_dir, exist_ok=True)
os.makedirs(scaler_save_dir, exist_ok=True)

# Iterate Through Each Stock
for stock in scaled_daily_data.keys():
    print(f"\n{'='*50}\nTraining LSTM Model for {stock}\n{'='*50}")
    
    # Retrieve Scaled Data
    data = scaled_daily_data[stock]
    X_train_scaled = data['X_train_scaled']
    X_test_scaled = data['X_test_scaled']
    y_train_scaled = data['y_train_scaled']
    y_test_scaled = data['y_test_scaled']
    scaler_X = data['scaler_X']
    scaler_y = data['scaler_y']
    
    # Create Sequences
    X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train_scaled, TIMESTEPS)
    X_test_seq, y_test_seq = create_sequences(X_test_scaled, y_test_scaled, TIMESTEPS)
    
    print(f" - Training sequences: {X_train_seq.shape}, Training targets: {y_train_seq.shape}")
    print(f" - Testing sequences: {X_test_seq.shape}, Testing targets: {y_test_seq.shape}")
    
    # Build the Model
    model = build_lstm_model(input_shape=(X_train_seq.shape[1], X_train_seq.shape[2]))
    model.summary()
    
    # Define Callbacks
    early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    checkpoint = ModelCheckpoint(
        filepath=os.path.join(model_save_dir, f'lstm_{stock}_best.keras'),  # Changed to .keras
        monitor='val_loss',
        save_best_only=True,
        verbose=1
    )
    
    # Train the Model
    history = model.fit(
        X_train_seq, y_train_seq,
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        validation_split=VALIDATION_SPLIT,
        callbacks=[early_stop, checkpoint],
        verbose=1
    )
    
    # Load the Best Model
    best_model_path = os.path.join(model_save_dir, f'lstm_{stock}_best.keras')
    model = load_model(best_model_path)
    print(f" - Loaded best model from {best_model_path}")
    
    # Predict on Test Data
    predictions_scaled = model.predict(X_test_seq).flatten()
    
    # Inverse Transform Predictions and Targets
    predictions = scaler_y.inverse_transform(predictions_scaled.reshape(-1, 1)).flatten()
    y_test = scaler_y.inverse_transform(y_test_seq.reshape(-1, 1)).flatten()
    
    # Ensure Consistent Lengths
    print(f" - Length of y_test: {len(y_test)}")
    print(f" - Length of predictions: {len(predictions)}")
    
    # Evaluation Metrics
    rmse, mae, r2 = evaluate_model(y_test, predictions)
    model_performance[stock] = {'RMSE': rmse, 'MAE': mae, 'R2': r2}
    
    print(f" - Evaluation Metrics for {stock}: RMSE = {rmse:.4f}, MAE = {mae:.4f}, R2 = {r2:.4f}")
    
    print(f"Model training and evaluation completed for {stock}.\n")

# Summary of Model Performance
print(f"\n{'='*50}\nSummary of Model Performance\n{'='*50}")
for stock, metrics in model_performance.items():
    print(f"{stock}: RMSE = {metrics['RMSE']:.4f}, MAE = {metrics['MAE']:.4f}, R2 = {metrics['R2']:.4f}")


In [ ]:
# Cell 7: Evaluation and Plotting of LSTM Models with Numerical Metrics Tables

TIMESTEPS = 60
model_save_dir = '../models/lstm_models'

def create_sequences(X, y, timesteps):
    X_seq, y_seq = [], []
    for i in range(timesteps, len(X)):
        X_seq.append(X[i-timesteps:i].values)
        y_seq.append(y[i])
    return np.array(X_seq), np.array(y_seq)

def calculate_group_metrics(eval_df, group_by):
    grouped = eval_df.groupby(group_by).apply(lambda x: pd.Series({
        'RMSE': np.sqrt(mean_squared_error(x['Actual'], x['Predicted'])),
        'MAE': mean_absolute_error(x['Actual'], x['Predicted']),
        'R2': r2_score(x['Actual'], x['Predicted']),
        'MAPE': mean_absolute_percentage_error(x['Actual'], x['Predicted']) * 100
    }), include_groups=False)
    return grouped

def plot_metrics_by_group(grouped_metrics, stock, grouping, metric):
    plt.figure(figsize=(10, 6))
    sns.barplot(x=grouped_metrics.index, y=grouped_metrics[metric])
    plt.title(f'{metric} by {grouping} for {stock}')
    plt.xlabel(grouping)
    plt.ylabel(metric)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

def plot_comparative_metrics(grouped_metrics, stocks, grouping, metric):
    plt.figure(figsize=(12, 8))
    for stock in stocks:
        sns.lineplot(data=grouped_metrics[stock][grouping], x=grouped_metrics[stock][grouping].index, y=metric, label=stock)
    plt.title(f'Comparison of {metric} across {grouping}')
    plt.xlabel(grouping)
    plt.ylabel(metric)
    plt.legend(title='Stock')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

def add_time_features(eval_df):
    eval_df['Month'] = eval_df['Date'].dt.month
    eval_df['Quarter'] = eval_df['Date'].dt.quarter
    eval_df['Season'] = eval_df['Month'].apply(lambda month: 'Winter' if month in [12, 1, 2] else 'Spring' if month in [3,4,5] else 'Summer' if month in [6,7,8] else 'Autumn')
    return eval_df

overall_metrics = {}
grouped_metrics_all = {'Month': {}, 'Quarter': {}, 'Season': {}}

for stock in scaled_daily_data.keys():
    print(f"\n{'='*50}\nEvaluating and Plotting LSTM Model for {stock}\n{'='*50}")
    
    data = scaled_daily_data[stock]
    X_test_scaled = data['X_test_scaled']
    y_test_scaled = data['y_test_scaled']
    scaler_y = data['scaler_y']
    
    X_test_seq, y_test_seq = create_sequences(X_test_scaled, y_test_scaled, TIMESTEPS)
    
    best_model_path = os.path.join(model_save_dir, f'lstm_{stock}_best.keras')
    if not os.path.exists(best_model_path):
        print(f" - Best model for {stock} not found at {best_model_path}. Skipping...")
        continue
    try:
        model = load_model(best_model_path)
    except Exception as e:
        print(f" - Failed to load model for {stock}: {e}. Skipping...")
        continue
    print(f" - Loaded best model from {best_model_path}")
    
    predictions_scaled = model.predict(X_test_seq).flatten()
    
    try:
        predictions = scaler_y.inverse_transform(predictions_scaled.reshape(-1, 1)).flatten()
        y_test = scaler_y.inverse_transform(y_test_seq.reshape(-1, 1)).flatten()
    except Exception as e:
        print(f" - Failed to inverse transform predictions or actuals for {stock}: {e}. Skipping...")
        continue
    
    if isinstance(X_test_scaled, pd.DataFrame) and isinstance(X_test_scaled.index, pd.DatetimeIndex):
        dates = X_test_scaled.index[TIMESTEPS:]
    elif isinstance(X_test_scaled, pd.DataFrame) and 'Date' in X_test_scaled.columns:
        dates = pd.to_datetime(X_test_scaled['Date'].iloc[TIMESTEPS:])
    else:
        print(f" - No Date information found for {stock}. Creating dummy dates.")
        dates = pd.date_range(start='2020-01-01', periods=len(y_test), freq='D')
    
    eval_df = pd.DataFrame({
        'Date': dates,
        'Actual': y_test,
        'Predicted': predictions
    })
    
    eval_df = add_time_features(eval_df)
    
    rmse = np.sqrt(mean_squared_error(eval_df['Actual'], eval_df['Predicted']))
    mae = mean_absolute_error(eval_df['Actual'], eval_df['Predicted'])
    r2 = r2_score(eval_df['Actual'], eval_df['Predicted'])
    mape = mean_absolute_percentage_error(eval_df['Actual'], eval_df['Predicted']) * 100
    
    overall_metrics[stock] = {
        'RMSE': rmse,
        'MAE': mae,
        'R2': r2,
        'MAPE': mape
    }
    
    print(f" - Overall Evaluation Metrics for {stock}:")
    print(f"    RMSE = {rmse:.4f}")
    print(f"    MAE = {mae:.4f}")
    print(f"    R2 = {r2:.4f}")
    print(f"    MAPE = {mape:.2f}%")
    
    grouped_metrics_month = calculate_group_metrics(eval_df, 'Month')
    grouped_metrics_quarter = calculate_group_metrics(eval_df, 'Quarter')
    grouped_metrics_season = calculate_group_metrics(eval_df, 'Season')
    
    grouped_metrics_all['Month'][stock] = grouped_metrics_month
    grouped_metrics_all['Quarter'][stock] = grouped_metrics_quarter
    grouped_metrics_all['Season'][stock] = grouped_metrics_season
    
    print(f" - Plotting RMSE by Season for {stock}")
    plot_metrics_by_group(grouped_metrics_season, stock, 'Season', 'RMSE')
    
    print(f" - Plotting MAE by Season for {stock}")
    plot_metrics_by_group(grouped_metrics_season, stock, 'Season', 'MAE')
    
    print(f" - Plotting R2 by Season for {stock}")
    plot_metrics_by_group(grouped_metrics_season, stock, 'Season', 'R2')
    
    print(f" - Plotting MAPE by Season for {stock}")
    plot_metrics_by_group(grouped_metrics_season, stock, 'Season', 'MAPE')
    
    print(f"Evaluation and plotting completed for {stock}.\n")

overall_metrics_df = pd.DataFrame(overall_metrics).T
print("\n" + "="*50)
print("Overall Evaluation Metrics for All Stocks")
print("="*50)
display(overall_metrics_df)
overall_metrics_df.to_csv('overall_evaluation_metrics.csv')
print("\n - Overall Evaluation Metrics table saved as 'overall_evaluation_metrics.csv'.")

def create_grouped_metrics_tables(grouped_metrics_all, grouping):
    grouped_metrics_tables = {}
    for stock, metrics in grouped_metrics_all[grouping].items():
        metrics_df = metrics.reset_index()
        metrics_df.columns = [grouping] + list(metrics_df.columns[1:])
        grouped_metrics_tables[stock] = metrics_df
    return grouped_metrics_tables

for grouping in ['Month', 'Quarter', 'Season']:
    grouped_tables = create_grouped_metrics_tables(grouped_metrics_all, grouping)
    for stock, table in grouped_tables.items():
        print(f"\n{'='*50}\n{grouping} Evaluation Metrics for {stock}\n{'='*50}")
        display(table)
        filename = f'{stock}_{grouping}_evaluation_metrics.csv'
        table.to_csv(filename, index=False)
        print(f" - {grouping} Evaluation Metrics table for {stock} saved as '{filename}'.")

for grouping in ['Month', 'Quarter', 'Season']:
    for metric in ['RMSE', 'MAE', 'R2', 'MAPE']:
        comparative_df = pd.DataFrame({stock: grouped_metrics_all[grouping][stock][metric] for stock in grouped_metrics_all[grouping].keys()})
        comparative_df.index.name = grouping
        print(f"\n{'='*50}\nComparative {metric} Across {grouping} for All Stocks\n{'='*50}")
        display(comparative_df)
        filename = f'comparative_{metric}_across_{grouping}.csv'
        comparative_df.to_csv(filename)
        print(f" - Comparative {metric} Across {grouping} table saved as '{filename}'.")


In [ ]:
# Cell 8: Training and Evaluating GRU Models for Daily Data

import os
import random
import numpy as np
import pandas as pd
import joblib
import logging
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import GRU, Dropout, Dense, Input
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import tensorflow as tf

# Define Parameters
TIMESTEPS = 60  # Number of past days to use for prediction
BATCH_SIZE = 32
EPOCHS = 100
VALIDATION_SPLIT = 0.1
SEED = 42

# Set random seeds for reproducibility
np.random.seed(SEED)
tf.random.set_seed(SEED)
random.seed(SEED)

# Function to Create Sequences (Already Defined in Cell 6)
def create_sequences(X, y, timesteps):
    X_seq, y_seq = [], []
    for i in range(timesteps, len(X)):
        X_seq.append(X[i-timesteps:i].values)
        y_seq.append(y[i])
    return np.array(X_seq), np.array(y_seq)

# Function to Build GRU Model
def build_gru_model(input_shape):
    model = Sequential()
    model.add(Input(shape=input_shape))
    model.add(GRU(units=50, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(GRU(units=50, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(units=25, activation='relu'))
    model.add(Dense(units=1))  # Output layer
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# Initialize a dictionary to store GRU model performance
gru_model_performance = {}

# Define Directory to Save GRU Models
gru_model_save_dir = '../models/gru_models'
os.makedirs(gru_model_save_dir, exist_ok=True)

# Iterate Through Each Stock
for stock in scaled_daily_data.keys():
    print(f"\n{'='*50}\nTraining GRU Model for {stock}\n{'='*50}")
    
    # Retrieve Scaled Data
    data = scaled_daily_data[stock]
    X_train_scaled = data['X_train_scaled']
    X_test_scaled = data['X_test_scaled']
    y_train_scaled = data['y_train_scaled']
    y_test_scaled = data['y_test_scaled']
    scaler_y = data['scaler_y']
    
    # Create Sequences
    X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train_scaled, TIMESTEPS)
    X_test_seq, y_test_seq = create_sequences(X_test_scaled, y_test_scaled, TIMESTEPS)
    
    print(f" - Training sequences: {X_train_seq.shape}, Training targets: {y_train_seq.shape}")
    print(f" - Testing sequences: {X_test_seq.shape}, Testing targets: {y_test_seq.shape}")
    
    # Build the GRU Model
    model = build_gru_model(input_shape=(X_train_seq.shape[1], X_train_seq.shape[2]))
    model.summary()
    
    # Define Callbacks
    early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    checkpoint = ModelCheckpoint(
        filepath=os.path.join(gru_model_save_dir, f'gru_{stock}_best.keras'),
        monitor='val_loss',
        save_best_only=True,
        verbose=1
    )
    
    # Train the GRU Model
    history = model.fit(
        X_train_seq, y_train_seq,
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        validation_split=VALIDATION_SPLIT,
        callbacks=[early_stop, checkpoint],
        verbose=1
    )
    
    # Load the Best Model
    best_model_path = os.path.join(gru_model_save_dir, f'gru_{stock}_best.keras')
    if os.path.exists(best_model_path):
        model = load_model(best_model_path)
        print(f" - Loaded best model from {best_model_path}")
    else:
        print(f" - Best GRU model for {stock} not found at {best_model_path}.")
        continue  # Skip evaluation if model not saved
    
    # Predict on Test Data
    predictions_scaled = model.predict(X_test_seq).flatten()
    
    # Inverse Transform Predictions and Targets
    predictions = scaler_y.inverse_transform(predictions_scaled.reshape(-1, 1)).flatten()
    y_test = scaler_y.inverse_transform(y_test_seq.reshape(-1, 1)).flatten()
    
    # Ensure Consistent Lengths
    print(f" - Length of y_test: {len(y_test)}")
    print(f" - Length of predictions: {len(predictions)}")
    
    # Evaluation Metrics
    rmse = np.sqrt(mean_squared_error(y_test, predictions))
    mae = mean_absolute_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)
    gru_model_performance[stock] = {'RMSE': rmse, 'MAE': mae, 'R2': r2}
    
    print(f" - Evaluation Metrics for GRU {stock}: RMSE = {rmse:.4f}, MAE = {mae:.4f}, R2 = {r2:.4f}")
    
    print(f"GRU model training and evaluation completed for {stock}.\n")

# Summary of GRU Model Performance
print(f"\n{'='*50}\nSummary of GRU Model Performance\n{'='*50}")
for stock, metrics in gru_model_performance.items():
    print(f"{stock}: RMSE = {metrics['RMSE']:.4f}, MAE = {metrics['MAE']:.4f}, R2 = {metrics['R2']:.4f}")


In [ ]:
# Cell 9: Training and Evaluating XGBoost Models

# Define Parameters
TIMESTEPS = 60  # Ensure consistency with LSTM
model_save_dir = '../models/xgb_models'
os.makedirs(model_save_dir, exist_ok=True)

# Initialize dictionaries to store metrics
overall_metrics_xgb = {}
grouped_metrics_all_xgb = {'Month': {}, 'Quarter': {}, 'Season': {}}

# Function to Add Time Features
def add_time_features_xgb(eval_df):
    eval_df['Month'] = eval_df['Date'].dt.month
    eval_df['Quarter'] = eval_df['Date'].dt.quarter
    eval_df['Season'] = eval_df['Month'].apply(
        lambda month: 'Winter' if month in [12, 1, 2] else
                      'Spring' if month in [3, 4, 5] else
                      'Summer' if month in [6, 7, 8] else
                      'Autumn'
    )
    return eval_df

# Iterate Through Each Stock for Evaluation and Plotting
for stock in scaled_daily_data.keys():
    print(f"\n{'='*50}\nTraining and Evaluating XGBoost Model for {stock}\n{'='*50}")
    
    # Retrieve Scaled Data
    data = scaled_daily_data[stock]
    X_train_scaled = data['X_train_scaled']
    y_train_scaled = data['y_train_scaled']
    X_test_scaled = data['X_test_scaled']
    y_test_scaled = data['y_test_scaled']
    scaler_y = data['scaler_y']

    # Initialize and Train XGBoost Regressor
    xgb_model = xgb.XGBRegressor(
        objective='reg:squarederror',
        n_estimators=100,
        learning_rate=0.1,
        max_depth=5,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        n_jobs=-1
    )
    
    xgb_model.fit(X_train_scaled, y_train_scaled)
    print(f" - XGBoost model trained for {stock}")
    
    # Predict on Test Data
    predictions_scaled = xgb_model.predict(X_test_scaled)
    
    # Inverse Transform Predictions and Targets
    predictions = scaler_y.inverse_transform(predictions_scaled.reshape(-1, 1)).flatten()
    y_test = scaler_y.inverse_transform(y_test_scaled.reshape(-1, 1)).flatten()
    
    # Create Evaluation DataFrame
    if isinstance(X_test_scaled, pd.DataFrame) and isinstance(X_test_scaled.index, pd.DatetimeIndex):
        dates = X_test_scaled.index
    elif isinstance(X_test_scaled, pd.DataFrame) and 'Date' in X_test_scaled.columns:
        dates = pd.to_datetime(X_test_scaled['Date'])
    else:
        print(f" - No Date information found for {stock}. Creating dummy dates.")
        dates = pd.date_range(start='2020-01-01', periods=len(y_test), freq='D')
    
    eval_df = pd.DataFrame({
        'Date': dates,
        'Actual': y_test,
        'Predicted': predictions
    })
    
    # Add Time Features
    eval_df = add_time_features_xgb(eval_df)
    
    # Calculate Overall Metrics
    rmse = np.sqrt(mean_squared_error(eval_df['Actual'], eval_df['Predicted']))
    mae = mean_absolute_error(eval_df['Actual'], eval_df['Predicted'])
    r2 = r2_score(eval_df['Actual'], eval_df['Predicted'])
    mape = mean_absolute_percentage_error(eval_df['Actual'], eval_df['Predicted']) * 100
    
    overall_metrics_xgb[stock] = {
        'RMSE': rmse,
        'MAE': mae,
        'R2': r2,
        'MAPE': mape
    }
    
    print(f" - Overall Evaluation Metrics for {stock}:")
    print(f"    RMSE = {rmse:.4f}")
    print(f"    MAE = {mae:.4f}")
    print(f"    R2 = {r2:.4f}")
    print(f"    MAPE = {mape:.2f}%")
    
    # Calculate Grouped Metrics
    grouped_metrics_month = eval_df.groupby('Month').apply(lambda x: pd.Series({
        'RMSE': np.sqrt(mean_squared_error(x['Actual'], x['Predicted'])),
        'MAE': mean_absolute_error(x['Actual'], x['Predicted']),
        'R2': r2_score(x['Actual'], x['Predicted']),
        'MAPE': mean_absolute_percentage_error(x['Actual'], x['Predicted']) * 100
    }), include_groups=False)
    
    grouped_metrics_quarter = eval_df.groupby('Quarter').apply(lambda x: pd.Series({
        'RMSE': np.sqrt(mean_squared_error(x['Actual'], x['Predicted'])),
        'MAE': mean_absolute_error(x['Actual'], x['Predicted']),
        'R2': r2_score(x['Actual'], x['Predicted']),
        'MAPE': mean_absolute_percentage_error(x['Actual'], x['Predicted']) * 100
    }), include_groups=False)
    
    grouped_metrics_season = eval_df.groupby('Season').apply(lambda x: pd.Series({
        'RMSE': np.sqrt(mean_squared_error(x['Actual'], x['Predicted'])),
        'MAE': mean_absolute_error(x['Actual'], x['Predicted']),
        'R2': r2_score(x['Actual'], x['Predicted']),
        'MAPE': mean_absolute_percentage_error(x['Actual'], x['Predicted']) * 100
    }), include_groups=False)
    
    grouped_metrics_all_xgb['Month'][stock] = grouped_metrics_month
    grouped_metrics_all_xgb['Quarter'][stock] = grouped_metrics_quarter
    grouped_metrics_all_xgb['Season'][stock] = grouped_metrics_season
    
    # Plot Metrics by Season
    plot_metrics_by_group(grouped_metrics_season, stock, 'Season', 'RMSE')
    plot_metrics_by_group(grouped_metrics_season, stock, 'Season', 'MAE')
    plot_metrics_by_group(grouped_metrics_season, stock, 'Season', 'R2')
    plot_metrics_by_group(grouped_metrics_season, stock, 'Season', 'MAPE')
    
    # Save the trained model
    model_save_path = os.path.join(model_save_dir, f'xgb_{stock}_model.json')
    xgb_model.save_model(model_save_path)
    print(f" - XGBoost model saved for {stock} at {model_save_path}")
    
    print(f"Training and evaluation completed for {stock}.\n")

# Create Overall Metrics Table
overall_metrics_xgb_df = pd.DataFrame(overall_metrics_xgb).T
print("\n" + "="*50)
print("Overall Evaluation Metrics for All Stocks - XGBoost")
print("="*50)
display(overall_metrics_xgb_df)
overall_metrics_xgb_df.to_csv('overall_evaluation_metrics_xgb.csv')
print("\n - Overall Evaluation Metrics table for XGBoost saved as 'overall_evaluation_metrics_xgb.csv'.")

# Function to Create Grouped Metrics Tables
def create_grouped_metrics_tables_xgb(grouped_metrics_all_xgb, grouping):
    grouped_metrics_tables_xgb = {}
    for stock, metrics in grouped_metrics_all_xgb[grouping].items():
        metrics_df = metrics.reset_index()
        metrics_df.columns = [grouping] + list(metrics_df.columns[1:])
        grouped_metrics_tables_xgb[stock] = metrics_df
    return grouped_metrics_tables_xgb

# Create and Save Grouped Metrics Tables
for grouping in ['Month', 'Quarter', 'Season']:
    grouped_tables_xgb = create_grouped_metrics_tables_xgb(grouped_metrics_all_xgb, grouping)
    for stock, table in grouped_tables_xgb.items():
        print(f"\n{'='*50}\n{grouping} Evaluation Metrics for {stock} - XGBoost\n{'='*50}")
        display(table)
        filename = f'{stock}_{grouping}_evaluation_metrics_xgb.csv'
        table.to_csv(filename, index=False)
        print(f" - {grouping} Evaluation Metrics table for {stock} saved as '{filename}'.")
    
    # Create Comparative Metrics Tables Across Stocks
    for metric in ['RMSE', 'MAE', 'R2', 'MAPE']:
        comparative_df_xgb = pd.DataFrame({stock: grouped_metrics_all_xgb[grouping][stock][metric] for stock in grouped_metrics_all_xgb[grouping].keys()})
        comparative_df_xgb.index.name = grouping
        print(f"\n{'='*50}\nComparative {metric} Across {grouping} for All Stocks - XGBoost\n{'='*50}")
        display(comparative_df_xgb)
        filename = f'comparative_{metric}_across_{grouping}_xgb.csv'
        comparative_df_xgb.to_csv(filename)
        print(f" - Comparative {metric} Across {grouping} table for XGBoost saved as '{filename}'.")


In [ ]:
# Cell 10: Training and Evaluating Random Forest Models

# Define Parameters
model_save_dir = '../models/random_forest_models'
os.makedirs(model_save_dir, exist_ok=True)

# Initialize dictionaries to store metrics
overall_metrics_rf = {}
grouped_metrics_all_rf = {'Month': {}, 'Quarter': {}, 'Season': {}}

# Function to Add Time Features (if not already added)
def add_time_features_rf(eval_df):
    eval_df['Month'] = eval_df['Date'].dt.month
    eval_df['Quarter'] = eval_df['Date'].dt.quarter
    eval_df['Season'] = eval_df['Month'].apply(
        lambda month: 'Winter' if month in [12, 1, 2] else
                      'Spring' if month in [3, 4, 5] else
                      'Summer' if month in [6, 7, 8] else
                      'Autumn'
    )
    return eval_df

# Iterate Through Each Stock for Evaluation and Plotting
for stock in scaled_daily_data.keys():
    print(f"\n{'='*50}\nTraining and Evaluating Random Forest Model for {stock}\n{'='*50}")
    
    # Retrieve Scaled Data
    data = scaled_daily_data[stock]
    X_train_scaled = data['X_train_scaled']
    y_train_scaled = data['y_train_scaled']
    X_test_scaled = data['X_test_scaled']
    y_test_scaled = data['y_test_scaled']
    scaler_y = data['scaler_y']
    
    # Initialize and Train Random Forest Regressor
    rf_model = RandomForestRegressor(
        n_estimators=100,
        max_depth=None,
        min_samples_split=2,
        min_samples_leaf=1,
        random_state=42,
        n_jobs=-1
    )
    
    rf_model.fit(X_train_scaled, y_train_scaled)
    print(f" - Random Forest model trained for {stock}")
    
    # Predict on Test Data
    predictions_scaled = rf_model.predict(X_test_scaled)
    
    # Inverse Transform Predictions and Targets
    predictions = scaler_y.inverse_transform(predictions_scaled.reshape(-1, 1)).flatten()
    y_test = scaler_y.inverse_transform(y_test_scaled.reshape(-1, 1)).flatten()
    
    # Create Evaluation DataFrame
    if isinstance(X_test_scaled, pd.DataFrame) and isinstance(X_test_scaled.index, pd.DatetimeIndex):
        dates = X_test_scaled.index
    elif isinstance(X_test_scaled, pd.DataFrame) and 'Date' in X_test_scaled.columns:
        dates = pd.to_datetime(X_test_scaled['Date'])
    else:
        print(f" - No Date information found for {stock}. Creating dummy dates.")
        dates = pd.date_range(start='2020-01-01', periods=len(y_test), freq='D')
    
    eval_df = pd.DataFrame({
        'Date': dates,
        'Actual': y_test,
        'Predicted': predictions
    })
    
    # Add Time Features
    eval_df = add_time_features_rf(eval_df)
    
    # Calculate Overall Metrics
    rmse = np.sqrt(mean_squared_error(eval_df['Actual'], eval_df['Predicted']))
    mae = mean_absolute_error(eval_df['Actual'], eval_df['Predicted'])
    r2 = r2_score(eval_df['Actual'], eval_df['Predicted'])
    mape = mean_absolute_percentage_error(eval_df['Actual'], eval_df['Predicted']) * 100
    
    overall_metrics_rf[stock] = {
        'RMSE': rmse,
        'MAE': mae,
        'R2': r2,
        'MAPE': mape
    }
    
    print(f" - Overall Evaluation Metrics for {stock}:")
    print(f"    RMSE = {rmse:.4f}")
    print(f"    MAE = {mae:.4f}")
    print(f"    R2 = {r2:.4f}")
    print(f"    MAPE = {mape:.2f}%")
    
    # Calculate Grouped Metrics
    grouped_metrics_month = eval_df.groupby('Month').apply(lambda x: pd.Series({
        'RMSE': np.sqrt(mean_squared_error(x['Actual'], x['Predicted'])),
        'MAE': mean_absolute_error(x['Actual'], x['Predicted']),
        'R2': r2_score(x['Actual'], x['Predicted']),
        'MAPE': mean_absolute_percentage_error(x['Actual'], x['Predicted']) * 100
    }), include_groups=False)
    
    grouped_metrics_quarter = eval_df.groupby('Quarter').apply(lambda x: pd.Series({
        'RMSE': np.sqrt(mean_squared_error(x['Actual'], x['Predicted'])),
        'MAE': mean_absolute_error(x['Actual'], x['Predicted']),
        'R2': r2_score(x['Actual'], x['Predicted']),
        'MAPE': mean_absolute_percentage_error(x['Actual'], x['Predicted']) * 100
    }), include_groups=False)
    
    grouped_metrics_season = eval_df.groupby('Season').apply(lambda x: pd.Series({
        'RMSE': np.sqrt(mean_squared_error(x['Actual'], x['Predicted'])),
        'MAE': mean_absolute_error(x['Actual'], x['Predicted']),
        'R2': r2_score(x['Actual'], x['Predicted']),
        'MAPE': mean_absolute_percentage_error(x['Actual'], x['Predicted']) * 100
    }), include_groups=False)
    
    grouped_metrics_all_rf['Month'][stock] = grouped_metrics_month
    grouped_metrics_all_rf['Quarter'][stock] = grouped_metrics_quarter
    grouped_metrics_all_rf['Season'][stock] = grouped_metrics_season
    
    # Plot Metrics by Season
    plot_metrics_by_group(grouped_metrics_season, stock, 'Season', 'RMSE')
    plot_metrics_by_group(grouped_metrics_season, stock, 'Season', 'MAE')
    plot_metrics_by_group(grouped_metrics_season, stock, 'Season', 'R2')
    plot_metrics_by_group(grouped_metrics_season, stock, 'Season', 'MAPE')
    
    # Save the trained model
    model_save_path = os.path.join(model_save_dir, f'rf_{stock}_model.pkl')
    joblib.dump(rf_model, model_save_path)
    print(f" - Random Forest model saved for {stock} at {model_save_path}")
    
    print(f"Training and evaluation completed for {stock}.\n")

# Create Overall Metrics Table
overall_metrics_rf_df = pd.DataFrame(overall_metrics_rf).T
print("\n" + "="*50)
print("Overall Evaluation Metrics for All Stocks - Random Forest")
print("="*50)
display(overall_metrics_rf_df)
overall_metrics_rf_df.to_csv('overall_evaluation_metrics_rf.csv')
print("\n - Overall Evaluation Metrics table for Random Forest saved as 'overall_evaluation_metrics_rf.csv'.")

# Function to Create Grouped Metrics Tables
def create_grouped_metrics_tables_rf(grouped_metrics_all_rf, grouping):
    grouped_metrics_tables_rf = {}
    for stock, metrics in grouped_metrics_all_rf[grouping].items():
        metrics_df = metrics.reset_index()
        metrics_df.columns = [grouping] + list(metrics_df.columns[1:])
        grouped_metrics_tables_rf[stock] = metrics_df
    return grouped_metrics_tables_rf

# Create and Save Grouped Metrics Tables
for grouping in ['Month', 'Quarter', 'Season']:
    grouped_tables_rf = create_grouped_metrics_tables_rf(grouped_metrics_all_rf, grouping)
    for stock, table in grouped_tables_rf.items():
        print(f"\n{'='*50}\n{grouping} Evaluation Metrics for {stock} - Random Forest\n{'='*50}")
        display(table)
        filename = f'{stock}_{grouping}_evaluation_metrics_rf.csv'
        table.to_csv(filename, index=False)
        print(f" - {grouping} Evaluation Metrics table for {stock} saved as '{filename}'.")
    
    # Create Comparative Metrics Tables Across Stocks
    for metric in ['RMSE', 'MAE', 'R2', 'MAPE']:
        comparative_df_rf = pd.DataFrame({stock: grouped_metrics_all_rf[grouping][stock][metric] for stock in grouped_metrics_all_rf[grouping].keys()})
        comparative_df_rf.index.name = grouping
        print(f"\n{'='*50}\nComparative {metric} Across {grouping} for All Stocks - Random Forest\n{'='*50}")
        display(comparative_df_rf)
        filename = f'comparative_{metric}_across_{grouping}_rf.csv'
        comparative_df_rf.to_csv(filename)
        print(f" - Comparative {metric} Across {grouping} table for Random Forest saved as '{filename}'.")


In [ ]:
# Cell 11: Developing a Stacking Meta-Model

# Define Paths for Models
xgb_model_dir = '../models/xgb_models'
rf_model_dir = '../models/random_forest_models'
lstm_model_dir = '../models/lstm_models'
gru_model_dir = '../models/gru_models'
meta_model_dir = '../models/meta_model'
os.makedirs(meta_model_dir, exist_ok=True)

# Initialize Dictionaries to Store Meta-Features and Targets
meta_features_train_dict = {}
meta_features_test_dict = {}
y_train_dict = {}
y_test_dict = {}

# Function to Create Sequences (Reused from Cell 6)
def create_sequences(X, y, timesteps):
    X_seq, y_seq = [], []
    for i in range(timesteps, len(X)):
        X_seq.append(X[i-timesteps:i].values)
        y_seq.append(y[i])
    return np.array(X_seq), np.array(y_seq)

# Iterate Through Each Stock to Populate Meta-Features
for stock in stocks:
    print(f"\n{'='*50}\nProcessing Stock: {stock}\n{'='*50}")
    
    # ----- Load Base Models -----
    # Load XGBoost Model
    xgb_model_path = os.path.join(xgb_model_dir, f'xgb_{stock}_model.json')
    if not os.path.exists(xgb_model_path):
        print(f" - XGBoost model for {stock} not found at {xgb_model_path}. Skipping.")
        continue
    xgb_model = xgb.XGBRegressor()
    xgb_model.load_model(xgb_model_path)
    print(f" - Loaded XGBoost model for {stock}.")
    
    # Load Random Forest Model
    rf_model_path = os.path.join(rf_model_dir, f'rf_{stock}_model.pkl')
    if not os.path.exists(rf_model_path):
        print(f" - Random Forest model for {stock} not found at {rf_model_path}. Skipping.")
        continue
    rf_model = joblib.load(rf_model_path)
    print(f" - Loaded Random Forest model for {stock}.")
    
    # Load LSTM Model
    lstm_model_path = os.path.join(lstm_model_dir, f'lstm_{stock}_best.keras')
    if not os.path.exists(lstm_model_path):
        print(f" - LSTM model for {stock} not found at {lstm_model_path}. Skipping.")
        continue
    lstm_model = load_model(lstm_model_path)
    print(f" - Loaded LSTM model for {stock}.")
    
    # Load GRU Model
    gru_model_path = os.path.join(gru_model_dir, f'gru_{stock}_best.keras')
    if not os.path.exists(gru_model_path):
        print(f" - GRU model for {stock} not found at {gru_model_path}. Skipping.")
        continue
    gru_model = load_model(gru_model_path)
    print(f" - Loaded GRU model for {stock}.")
    
    # ----- Retrieve Scaled Data -----
    data = scaled_daily_data.get(stock)
    if data is None:
        print(f" - No scaled data found for {stock}. Skipping.")
        continue
    
    X_train_scaled = data['X_train_scaled']
    X_test_scaled = data['X_test_scaled']
    y_train_scaled = data['y_train_scaled']
    y_test_scaled = data['y_test_scaled']
    scaler_y = data['scaler_y']
    
    # ----- Create Sequences for LSTM and GRU Models -----
    TIMESTEPS = 60  # Ensure consistency
    X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train_scaled, TIMESTEPS)
    X_test_seq, y_test_seq = create_sequences(X_test_scaled, y_test_scaled, TIMESTEPS)
    
    print(f" - Training sequences: {X_train_seq.shape}, Training targets: {y_train_seq.shape}")
    print(f" - Testing sequences: {X_test_seq.shape}, Testing targets: {y_test_seq.shape}")
    
    # ----- Generate Predictions from Base Models -----
    # XGBoost Predictions
    xgb_pred_train_scaled = xgb_model.predict(X_train_scaled.iloc[TIMESTEPS:])
    xgb_pred_test_scaled = xgb_model.predict(X_test_scaled.iloc[TIMESTEPS:])
    xgb_pred_train = scaler_y.inverse_transform(xgb_pred_train_scaled.reshape(-1, 1)).flatten()
    xgb_pred_test = scaler_y.inverse_transform(xgb_pred_test_scaled.reshape(-1, 1)).flatten()
    
    # Random Forest Predictions
    rf_pred_train_scaled = rf_model.predict(X_train_scaled.iloc[TIMESTEPS:])
    rf_pred_test_scaled = rf_model.predict(X_test_scaled.iloc[TIMESTEPS:])
    rf_pred_train = scaler_y.inverse_transform(rf_pred_train_scaled.reshape(-1, 1)).flatten()
    rf_pred_test = scaler_y.inverse_transform(rf_pred_test_scaled.reshape(-1, 1)).flatten()
    
    # LSTM Predictions
    lstm_pred_train_scaled = lstm_model.predict(X_train_seq).flatten()
    lstm_pred_test_scaled = lstm_model.predict(X_test_seq).flatten()
    lstm_pred_train = scaler_y.inverse_transform(lstm_pred_train_scaled.reshape(-1, 1)).flatten()
    lstm_pred_test = scaler_y.inverse_transform(lstm_pred_test_scaled.reshape(-1, 1)).flatten()
    
    # GRU Predictions
    gru_pred_train_scaled = gru_model.predict(X_train_seq).flatten()
    gru_pred_test_scaled = gru_model.predict(X_test_seq).flatten()
    gru_pred_train = scaler_y.inverse_transform(gru_pred_train_scaled.reshape(-1, 1)).flatten()
    gru_pred_test = scaler_y.inverse_transform(gru_pred_test_scaled.reshape(-1, 1)).flatten()
    
    # ----- Align Predictions -----
    min_length_train = min(len(xgb_pred_train), len(rf_pred_train), len(lstm_pred_train), len(gru_pred_train))
    min_length_test = min(len(xgb_pred_test), len(rf_pred_test), len(lstm_pred_test), len(gru_pred_test))
    
    xgb_pred_train = xgb_pred_train[:min_length_train]
    rf_pred_train = rf_pred_train[:min_length_train]
    lstm_pred_train = lstm_pred_train[:min_length_train]
    gru_pred_train = gru_pred_train[:min_length_train]
    y_train = scaler_y.inverse_transform(y_train_seq.reshape(-1, 1)).flatten()[:min_length_train]
    
    xgb_pred_test = xgb_pred_test[:min_length_test]
    rf_pred_test = rf_pred_test[:min_length_test]
    lstm_pred_test = lstm_pred_test[:min_length_test]
    gru_pred_test = gru_pred_test[:min_length_test]
    y_test = scaler_y.inverse_transform(y_test_seq.reshape(-1, 1)).flatten()[:min_length_test]
    
    # ----- Populate Meta-Features for Training Data -----
    meta_features_train = pd.DataFrame({
        'XGB_Pred': xgb_pred_train,
        'RF_Pred': rf_pred_train,
        'LSTM_Pred': lstm_pred_train,
        'GRU_Pred': gru_pred_train
    })
    meta_features_train_dict[stock] = meta_features_train
    print(f" - Meta-features for training data populated for {stock}.")
    
    # ----- Populate Meta-Features for Test Data -----
    meta_features_test = pd.DataFrame({
        'XGB_Pred': xgb_pred_test,
        'RF_Pred': rf_pred_test,
        'LSTM_Pred': lstm_pred_test,
        'GRU_Pred': gru_pred_test
    })
    meta_features_test_dict[stock] = meta_features_test
    print(f" - Meta-features for testing data populated for {stock}.")
    
    # ----- Store Target Variables -----
    y_train_dict[stock] = y_train
    y_test_dict[stock] = y_test
    print(f" - Target variables stored for {stock}.")

# ----- Check Meta-Features DataFrames -----
print("\n" + "="*50 + "\nMeta-Features DataFrame Shapes\n" + "="*50)
for stock in meta_features_train_dict.keys():
    print(f" - {stock}: meta_features_train shape: {meta_features_train_dict[stock].shape}, meta_features_test shape: {meta_features_test_dict[stock].shape}, y_train shape: {y_train_dict[stock].shape}, y_test shape: {y_test_dict[stock].shape}")

# ----- Train Meta-Models for Each Stock -----
print("\n" + "="*50 + "\nTraining Meta-Models for Each Stock\n" + "="*50)

meta_model_per_stock = {}

for stock in meta_features_train_dict.keys():
    print(f"\nTraining Meta-Model for {stock}")
    
    meta_features_train = meta_features_train_dict[stock]
    meta_features_test = meta_features_test_dict[stock]
    y_train = y_train_dict[stock]
    y_test = y_test_dict[stock]
    
    # Check if meta_features_train and y_train are non-empty
    if meta_features_train.empty or len(y_train) == 0:
        print(f" - Empty meta-features or target variables for {stock}. Skipping.")
        continue
    
    # Train Meta-Model (Ridge Regression with Cross-Validation)
    meta_model = RidgeCV()
    meta_model.fit(meta_features_train, y_train)
    print(f" - Meta-Model (Ridge Regression) trained successfully for {stock}.")
    
    # Save Meta-Model
    meta_model_path = os.path.join(meta_model_dir, f'stacking_meta_model_{stock}.pkl')
    joblib.dump(meta_model, meta_model_path)
    print(f" - Meta-Model saved at '{meta_model_path}'")
    
    # ----- Generate Meta-Predictions on Test Data -----
    meta_pred_test = meta_model.predict(meta_features_test)
    
    # ----- Evaluate Meta-Model -----
    rmse_meta = np.sqrt(mean_squared_error(y_test, meta_pred_test))
    mae_meta = mean_absolute_error(y_test, meta_pred_test)
    r2_meta = r2_score(y_test, meta_pred_test)
    mape_meta = mean_absolute_percentage_error(y_test, meta_pred_test) * 100
    
    print(f"\nMeta-Model Evaluation Metrics for {stock}:")
    print(f"    RMSE = {rmse_meta:.4f}")
    print(f"    MAE = {mae_meta:.4f}")
    print(f"    R2 = {r2_meta:.4f}")
    print(f"    MAPE = {mape_meta:.2f}%")
    
    # ----- Create Evaluation DataFrame for Meta-Model -----
    # Assuming 'Date' is aligned with the test data's index
    X_test_scaled = scaled_daily_data[stock]['X_test_scaled']
    if isinstance(X_test_scaled, pd.DataFrame) and isinstance(X_test_scaled.index, pd.DatetimeIndex):
        dates = X_test_scaled.index
    elif isinstance(X_test_scaled, pd.DataFrame) and 'Date' in X_test_scaled.columns:
        dates = pd.to_datetime(X_test_scaled['Date'], errors='coerce')
    else:
        print(f" - No Date information found for {stock}. Creating dummy dates.")
        if len(meta_pred_test) > 0:
            last_date = X_test_scaled.index[-1] if isinstance(X_test_scaled.index, pd.DatetimeIndex) else pd.Timestamp('2020-01-01')
            dates = pd.date_range(start=last_date + pd.Timedelta(days=1), periods=len(meta_pred_test), freq='D')
        else:
            dates = pd.date_range(start='2020-01-01', periods=len(meta_pred_test), freq='D')
    
    eval_df_meta = pd.DataFrame({
        'Date': dates,
        'Actual': y_test,
        'Meta_Predicted': meta_pred_test
    })
    
    # ----- Add Time Features -----
    def add_time_features_meta(eval_df):
        # Ensure 'Date' is in datetime format
        if 'Date' not in eval_df.columns:
            raise KeyError("The DataFrame does not contain a 'Date' column.")
        
        if not pd.api.types.is_datetime64_any_dtype(eval_df['Date']):
            eval_df['Date'] = pd.to_datetime(eval_df['Date'], errors='coerce')
        
        # Check for any NaT values after conversion
        if eval_df['Date'].isnull().any():
            print("Warning: Some 'Date' entries could not be converted to datetime and will be dropped.")
            eval_df = eval_df.dropna(subset=['Date'])
        
        eval_df['Month'] = eval_df['Date'].dt.month
        eval_df['Quarter'] = eval_df['Date'].dt.quarter
        eval_df['Season'] = eval_df['Month'].apply(
            lambda month: 'Winter' if month in [12, 1, 2] else
                          'Spring' if month in [3, 4, 5] else
                          'Summer' if month in [6, 7, 8] else
                          'Autumn'
        )
        return eval_df
    
    try:
        eval_df_meta = add_time_features_meta(eval_df_meta)
        print(f" - Time features added to the meta-model evaluation DataFrame for {stock}.")
    except KeyError as e:
        print(f"Error: {e}")
    
    # ----- Save Meta-Model Predictions (Optional) -----
    meta_pred_save_path = os.path.join(meta_model_dir, f'meta_predictions_{stock}.csv')
    eval_df_meta.to_csv(meta_pred_save_path, index=False)
    print(f" - Meta-Model predictions saved at '{meta_pred_save_path}'")
    
    # ----- Store Meta-Model Performance -----
    meta_model_per_stock[stock] = {
        'RMSE': rmse_meta,
        'MAE': mae_meta,
        'R2': r2_meta,
        'MAPE': mape_meta
    }

# ----- Create Overall Metrics Table -----
overall_metrics_meta_df = pd.DataFrame(meta_model_per_stock).T
print("\n" + "="*50)
print("Overall Evaluation Metrics for All Stocks - Meta Stacked Model")
print("="*50)
display(overall_metrics_meta_df)
overall_metrics_meta_df.to_csv('overall_evaluation_metrics_meta_stacked.csv')
print("\n - Overall Evaluation Metrics table for Meta Stacked Model saved as 'overall_evaluation_metrics_meta_stacked.csv'.")


In [ ]:
# Cell 12A: Define Essential Functions

# Suppress specific warnings for cleaner output
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

def load_models(stock):
    """
    Load base models and meta-model for a given stock.
    """
    try:
        # Define model directories
        xgb_model_dir = '../models/xgb_models'
        rf_model_dir = '../models/random_forest_models'
        lstm_model_dir = '../models/lstm_models'
        gru_model_dir = '../models/gru_models'
        meta_model_dir = '../models/meta_model'
        scalers_dir = '../models/scalers'
        
        # Load XGBoost Model
        xgb_model_path = os.path.join(xgb_model_dir, f'xgb_{stock}_model.json')
        xgb_model = xgb.XGBRegressor()
        xgb_model.load_model(xgb_model_path)
        print(f" - Loaded XGBoost model from '{xgb_model_path}'")
        
        # Load Random Forest Model
        rf_model_path = os.path.join(rf_model_dir, f'rf_{stock}_model.pkl')
        rf_model = joblib.load(rf_model_path)
        print(f" - Loaded Random Forest model from '{rf_model_path}'")
        
        # Load LSTM Model
        lstm_model_path = os.path.join(lstm_model_dir, f'lstm_{stock}_best.keras')
        lstm_model = load_model(lstm_model_path)
        print(f" - Loaded LSTM model from '{lstm_model_path}'")
        
        # Load GRU Model
        gru_model_path = os.path.join(gru_model_dir, f'gru_{stock}_best.keras')
        gru_model = load_model(gru_model_path)
        print(f" - Loaded GRU model from '{gru_model_path}'")
        
        # Load Meta-Model
        meta_model_path = os.path.join(meta_model_dir, f'stacking_meta_model_{stock}.pkl')
        meta_model = joblib.load(meta_model_path)
        print(f" - Loaded Meta-Model from '{meta_model_path}'")
        
        # Load Scalers
        scaler_X_path = os.path.join(scalers_dir, f'minmax_scaler_X_{stock}.joblib')
        scaler_y_path = os.path.join(scalers_dir, f'minmax_scaler_y_{stock}.joblib')
        scaler_X = joblib.load(scaler_X_path)
        scaler_y = joblib.load(scaler_y_path)
        print(f" - Loaded Scalers from '{scaler_X_path}' and '{scaler_y_path}'")
        
        return xgb_model, rf_model, lstm_model, gru_model, meta_model, scaler_X, scaler_y
    except Exception as e:
        print(f" - Error loading models or scalers for {stock}: {e}")
        return None

def calculate_vwap(df):
    """Calculate Volume Weighted Average Price (VWAP)."""
    typical_price = (df['High'] + df['Low'] + df['Close']) / 3
    vwap = (typical_price * df['Volume']).cumsum() / df['Volume'].cumsum()
    return vwap

def add_basic_technical_indicators(df):
    """
    Add basic technical indicators to the DataFrame.
    """
    # Simple and Exponential Moving Averages
    df['SMA_20'] = ta.trend.SMAIndicator(close=df['Close'], window=20, fillna=False).sma_indicator()
    df['EMA_20'] = ta.trend.EMAIndicator(close=df['Close'], window=20, fillna=False).ema_indicator()

    # Relative Strength Index
    df['RSI_14'] = ta.momentum.RSIIndicator(close=df['Close'], window=14, fillna=False).rsi()

    # Moving Average Convergence Divergence
    macd = ta.trend.MACD(close=df['Close'], window_slow=26, window_fast=12, window_sign=9, fillna=False)
    df['MACD'] = macd.macd()
    df['MACD_signal'] = macd.macd_signal()
    df['MACD_diff'] = macd.macd_diff()

    # Average True Range
    df['ATR_14'] = ta.volatility.AverageTrueRange(
        high=df['High'], low=df['Low'], close=df['Close'], window=14, fillna=False
    ).average_true_range()

    # Bollinger Bands
    bollinger = ta.volatility.BollingerBands(close=df['Close'], window=20, window_dev=2, fillna=False)
    df['Bollinger_High'] = bollinger.bollinger_hband()
    df['Bollinger_Low'] = bollinger.bollinger_lband()
    df['Bollinger_Width'] = bollinger.bollinger_wband()
    
    # On-Balance Volume
    df['OBV'] = ta.volume.OnBalanceVolumeIndicator(close=df['Close'], volume=df['Volume'], fillna=False).on_balance_volume()
    
    # Stochastic Oscillator
    stoch = ta.momentum.StochasticOscillator(
        high=df['High'], low=df['Low'], close=df['Close'], window=14, smooth_window=3, fillna=False
    )
    df['Stoch_%K'] = stoch.stoch()
    df['Stoch_%D'] = stoch.stoch_signal()

    # Rate of Change
    df['ROC_10'] = ta.momentum.ROCIndicator(close=df['Close'], window=10, fillna=False).roc()
    
    # Daily Returns
    df['Daily_Return'] = df['Close'].pct_change()
    
    # Close-Open Percent
    df['Close_Open_Percent'] = (df['Close'] - df['Open']) / df['Open'] * 100
    
    # High-Low Percent
    df['High_Low_Percent'] = (df['High'] - df['Low']) / df['Low'] * 100

    # Fill any NaN values
    df.fillna(method='bfill', inplace=True)
    df.fillna(method='ffill', inplace=True)

    return df

def add_advanced_features_daily(df):
    """
    Add advanced features to the DataFrame.
    """
    # a. Relative Strength Index (RSI_14)
    # Already added in basic indicators; skip if duplicate

    # b. Moving Average Convergence Divergence (MACD)
    # Already added in basic indicators; skip if duplicate

    # c. Average True Range (ATR_14)
    # Already added in basic indicators; skip if duplicate

    # d. Bollinger Bands Width (Bollinger_Width)
    # Already added in basic indicators; skip if duplicate

    # e. Keltner Channels
    keltner = ta.volatility.KeltnerChannel(
        high=df['High'], low=df['Low'], close=df['Close'], window=20, window_atr=10, fillna=False
    )
    df['Keltner_High'] = keltner.keltner_channel_hband()
    df['Keltner_Low'] = keltner.keltner_channel_lband()
    df['Keltner_Width'] = df['Keltner_High'] - df['Keltner_Low']

    # f. Chaikin Money Flow (CMF)
    cmf_indicator = ta.volume.ChaikinMoneyFlowIndicator(
        high=df['High'], low=df['Low'], close=df['Close'], volume=df['Volume'], window=20, fillna=False
    )
    df['CMF'] = cmf_indicator.chaikin_money_flow()

    # 2. Lag Features
    for lag in range(1, 4):
        df[f'Lag_Close_{lag}'] = df['Close'].shift(lag)

    # 3. Moving Averages and Standard Deviations
    df['MA_5'] = df['Close'].rolling(window=5, min_periods=5).mean()
    df['MA_Volume_10'] = df['Volume'].rolling(window=10, min_periods=10).mean()
    df['STD_Close_5'] = df['Close'].rolling(window=5, min_periods=5).std()
    df['MA_RSI_10'] = df['RSI_14'].rolling(window=10, min_periods=10).mean()
    df['MA_MACD_10'] = df['MACD'].rolling(window=10, min_periods=10).mean()
    df['MA_ATR_10'] = df['ATR_14'].rolling(window=10, min_periods=10).mean()
    df['MA_Bollinger_Width_10'] = df['Bollinger_Width'].rolling(window=10, min_periods=10).mean()

    # 4. VWAP and its Moving Average
    df['VWAP'] = calculate_vwap(df)
    df['MA_VWAP_10'] = df['VWAP'].rolling(window=10, min_periods=10).mean()

    # 5. Date Features
    if 'Date' not in df.columns:
        raise KeyError("Missing 'Date' column required for date-based feature engineering.")
    df['Day_of_Week'] = df['Date'].dt.dayofweek
    df['Month'] = df['Date'].dt.month

    # 6. Rolling Correlations and Momentum
    df['Rolling_Corr_Close_Volume_10'] = df['Close'].rolling(window=10, min_periods=10).corr(df['Volume'])
    df['Momentum_5'] = df['Close'].diff(5)
    df['Volatility_10'] = df['Close'].rolling(window=10, min_periods=10).std()

    # 7. Cyclical Encoding for Day of Week and Month
    df['Day_of_Week_sin'] = np.sin(2 * np.pi * df['Day_of_Week'] / 7)
    df['Day_of_Week_cos'] = np.cos(2 * np.pi * df['Day_of_Week'] / 7)
    df['Month_sin'] = np.sin(2 * np.pi * df['Month'] / 12)
    df['Month_cos'] = np.cos(2 * np.pi * df['Month'] / 12)

    # Drop original Day_of_Week and Month
    df.drop(['Day_of_Week', 'Month'], axis=1, inplace=True)

    # 8. Interaction Features
    df['MACD_diff_RSI_14'] = df['MACD_diff'] * df['RSI_14']
    df['MACD_diff_ATR_14'] = df['MACD_diff'] * df['ATR_14']
    df['RSI_14_ATR_14'] = df['RSI_14'] * df['ATR_14']

    # 9. Trend and Seasonal Components
    df['Trend'] = df['Close'].rolling(window=30, min_periods=30).mean()
    df['Seasonal'] = df['Close'] - df['Trend']

    # 10. Rolling Skewness and Kurtosis
    df['Rolling_Skew_Close_10'] = df['Close'].rolling(window=10, min_periods=10).skew()
    df['Rolling_Kurt_Close_10'] = df['Close'].rolling(window=10, min_periods=10).kurt()

    # Fill any remaining NaN values
    df.fillna(method='bfill', inplace=True)
    df.fillna(method='ffill', inplace=True)

    return df

def generate_next_day_features(current_data, scaler_X, timesteps=60):
    """
    Generate features for the next day forecast.
    """
    if len(current_data) < timesteps:
        raise ValueError("Insufficient data to generate features.")
    
    latest_data = current_data.copy()
    if 'Close' not in latest_data.columns:
        raise KeyError("'Close' column is missing in current_data.")
    
    # Apply basic and advanced feature engineering
    latest_data = add_basic_technical_indicators(latest_data)
    latest_data = add_advanced_features_daily(latest_data)
    
    # Drop 'Close' and 'Date' to get feature columns
    if 'Date' in latest_data.columns:
        features_for_models = latest_data.drop(columns=['Close', 'Date'])
    else:
        features_for_models = latest_data.drop(columns=['Close'])
    
    # Ensure features are in the same order as during training
    expected_features = scaler_X.feature_names_in_
    features_for_models = features_for_models[expected_features]
    
    # Verify feature count matches scaler's expectation
    actual_features = features_for_models.shape[1]
    if actual_features != len(expected_features):
        raise ValueError(f"Feature count mismatch: expected {len(expected_features)}, got {actual_features}")
    
    # For XGBoost and Random Forest: use the latest timestep's features
    latest_timestep = features_for_models.iloc[-1:].copy()  # DataFrame with 1 row
    xgb_rf_features_scaled = scaler_X.transform(latest_timestep)
    
    # For LSTM and GRU: use the entire sequence of timesteps
    lstm_gru_features = features_for_models.tail(timesteps).copy()
    lstm_gru_features_scaled = scaler_X.transform(lstm_gru_features).reshape(1, timesteps, -1)
    
    # Return the scaled features and the latest features for appending
    latest_features = latest_timestep.iloc[0].copy()
    return xgb_rf_features_scaled, lstm_gru_features_scaled, latest_features


In [ ]:
# Cell 12B: Initialize and Prepare Data

# Define Paths
raw_data_dir = '../data/stock_data'  # Directory containing raw CSVs
scaled_data_dir = '../models/scalers'  # Directory containing scalers
forecast_save_dir = '../models/future_forecasts'
os.makedirs(forecast_save_dir, exist_ok=True)

# Initialize a dictionary to store models and scaled data for each stock
models_per_stock = {}
timesteps = 60  # Number of timesteps used in LSTM/GRU models

for stock in stocks:
    print(f"\n{'='*50}\nProcessing Stock: {stock}\n{'='*50}")
    
    # Load models
    models = load_models(stock)
    if models is None:
        print(f" - Skipping stock '{stock}' due to model loading issues.")
        continue
    xgb_model, rf_model, lstm_model, gru_model, meta_model, scaler_X, scaler_y = models
    
    # Load raw data
    raw_csv_path = os.path.join(raw_data_dir, f"{stock}_daily.csv")
    if not os.path.exists(raw_csv_path):
        print(f" - Raw data CSV not found at '{raw_csv_path}'. Skipping.")
        continue
    
    try:
        df = pd.read_csv(raw_csv_path)
        print(f" - Loaded raw data from '{raw_csv_path}'. Shape: {df.shape}")
    except Exception as e:
        print(f" - Error reading CSV for {stock}: {e}")
        continue
    
    # Ensure 'Date' column is present
    if 'Date' not in df.columns:
        print(f" - 'Date' column missing in '{raw_csv_path}'. Skipping.")
        continue
    
    # Convert 'Date' to datetime
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
    if df['Date'].isnull().any():
        print(f" - Some 'Date' entries could not be converted to datetime for {stock}. Dropping these rows.")
        df.dropna(subset=['Date'], inplace=True)
    
    # Sort by Date
    df.sort_values('Date', inplace=True)
    df.reset_index(drop=True, inplace=True)
    
    # Add Basic Technical Indicators
    try:
        df = add_basic_technical_indicators(df)
        print(f" - Added basic technical indicators. Shape: {df.shape}")
    except Exception as e:
        print(f" - Error during adding basic technical indicators for {stock}: {e}")
        continue
    
    # Add Advanced Features
    try:
        df_fe = add_advanced_features_daily(df)
        print(f" - Applied advanced feature engineering. Shape: {df_fe.shape}")
    except Exception as e:
        print(f" - Error during advanced feature engineering for {stock}: {e}")
        continue
    
    # Keep only necessary recent data
    current_data = df_fe.copy()
    
    # Prepare features
    feature_cols = [col for col in current_data.columns if col not in ['Date', 'Close']]
    X_current = current_data[feature_cols].copy()
    
    # Ensure features are in the same order as during training
    expected_features = scaler_X.feature_names_in_
    missing_features = [feat for feat in expected_features if feat not in X_current.columns]
    if missing_features:
        print(f" - Missing features in current data for {stock}: {missing_features}")
        continue
    X_current = X_current[expected_features]
    
    # Scale features and target
    try:
        X_current_scaled = pd.DataFrame(scaler_X.transform(X_current), columns=X_current.columns)
        current_data_scaled = X_current_scaled.copy()
        current_data_scaled['Close'] = scaler_y.transform(current_data['Close'].values.reshape(-1, 1)).flatten()
        current_data_scaled['Date'] = current_data['Date'].values
        print(f" - Scaled current data for {stock}")
    except Exception as e:
        print(f" - Error scaling data for {stock}: {e}")
        continue
    
    # Store prepared data in models_per_stock
    models_per_stock[stock] = {
        'models': models,
        'current_data': current_data_scaled.tail(timesteps).copy(),  # Keep the last 'timesteps' rows
        'scaler_y': scaler_y
    }
    
    print(f" - Current data for {stock} loaded and prepared. Total samples: {len(current_data_scaled)}")


In [ ]:
# Cell 13A: Forecasting Setup and Initialization

# Import necessary libraries (if not already imported)
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates

# Ensure the forecast directory exists
forecast_save_dir = '../models/future_forecasts'
os.makedirs(forecast_save_dir, exist_ok=True)

# Forecast parameters
forecast_days = 30  # Number of days to forecast

# Initialize a dictionary to store forecast results for each stock
forecast_results = {stock: {} for stock in stocks}

# Function to check for NaN or Inf values in DataFrame
def check_nan_inf(df, day, stock):
    if df.isnull().values.any():
        print(f"   - Warning: NaN values found in features on Day {day} for {stock}")
        print(df.isnull().sum())
        return True
    if np.isinf(df.values).any():
        print(f"   - Warning: Inf values found in features on Day {day} for {stock}")
        return True
    return False

print("Forecasting setup completed. Ready to proceed to forecasting in Cell 13B.")


In [ ]:
# Cell 13B: Perform Recursive Forecasting and Display Results

for stock in stocks:
    print(f"\n{'='*50}\nStarting Forecasting for {stock}\n{'='*50}")
    
    # Retrieve models and current_data from models_per_stock
    stock_data = models_per_stock.get(stock)
    if stock_data is None:
        print(f" - No prepared data for '{stock}'. Skipping.")
        continue
    
    models = stock_data['models']
    current_data = stock_data['current_data'].copy()
    scaler_y = stock_data['scaler_y']
    
    xgb_model, rf_model, lstm_model, gru_model, meta_model, scaler_X, scaler_y = models
    
    # Initialize the last date
    last_date = current_data['Date'].iloc[-1]
    print(f" - Last historical date: {last_date}")
    
    # Initialize lists to store individual model predictions and dates
    xgb_predictions = []
    rf_predictions = []
    lstm_predictions = []
    gru_predictions = []
    meta_predictions = []
    forecasted_dates = []
    
    for day in range(1, forecast_days + 1):
        try:
            # Generate features for the next day
            xgb_rf_scaled, lstm_gru_scaled, latest_features = generate_next_day_features(current_data, scaler_X)
            
            # Predictions
            xgb_pred_scaled = xgb_model.predict(xgb_rf_scaled)
            xgb_pred = scaler_y.inverse_transform(xgb_pred_scaled.reshape(-1, 1)).flatten()[0]
            
            rf_pred_scaled = rf_model.predict(xgb_rf_scaled)
            rf_pred = scaler_y.inverse_transform(rf_pred_scaled.reshape(-1, 1)).flatten()[0]
            
            lstm_pred_scaled = lstm_model.predict(lstm_gru_scaled)
            lstm_pred = scaler_y.inverse_transform(lstm_pred_scaled.reshape(-1, 1)).flatten()[0]
            
            gru_pred_scaled = gru_model.predict(lstm_gru_scaled)
            gru_pred = scaler_y.inverse_transform(gru_pred_scaled.reshape(-1, 1)).flatten()[0]
            
            # Meta-Model Prediction
            meta_features = pd.DataFrame({
                'XGB_Pred': [xgb_pred],
                'RF_Pred': [rf_pred],
                'LSTM_Pred': [lstm_pred],
                'GRU_Pred': [gru_pred]
            })
            meta_pred = meta_model.predict(meta_features)[0]
            
            # Append predictions and dates
            xgb_predictions.append(xgb_pred)
            rf_predictions.append(rf_pred)
            lstm_predictions.append(lstm_pred)
            gru_predictions.append(gru_pred)
            meta_predictions.append(meta_pred)
            forecasted_dates.append(last_date + pd.Timedelta(days=1))
            
            print(f"   - Day {day}: Meta-Model Predicted Price = {meta_pred:.2f}")
            
            # Append the forecasted 'Close' and 'Date' to current_data
            new_close_scaled = scaler_y.transform([[meta_pred]])[0][0]  # Transform to scaled value
            new_date = last_date + pd.Timedelta(days=1)
            
            # Create a new row with updated 'Close' and 'Date'
            new_row = latest_features.copy()
            new_row['Close'] = new_close_scaled
            new_row['Date'] = new_date
            
            # Convert Series to DataFrame
            new_row_df = new_row.to_frame().T
            
            # Concatenate the new row to current_data
            current_data = pd.concat([current_data, new_row_df], ignore_index=True)
            
            # Update last_date
            last_date = new_date
            
            # Keep only the last 'timesteps' rows
            if len(current_data) > timesteps:
                current_data = current_data.iloc[-timesteps:].copy()
            
            # Recompute features for current_data after adding the new row
            # Inverse transform 'Close' back to original scale for feature computation
            current_data['Close'] = scaler_y.inverse_transform(current_data['Close'].values.reshape(-1, 1)).flatten()
            # Ensure 'Date' is in datetime format
            current_data['Date'] = pd.to_datetime(current_data['Date'])
            
            # Recompute features
            current_data_with_features = current_data.copy()
            current_data_with_features = add_basic_technical_indicators(current_data_with_features)
            current_data_with_features = add_advanced_features_daily(current_data_with_features)
            
            # **Debugging: Print data types and sample data**
            print(f"   - Data types of features on Day {day} for {stock}:\n{current_data_with_features.dtypes}")
            print(f"   - Sample data on Day {day} for {stock}:\n{current_data_with_features.tail(3)}")
            
            # Ensure numeric data types
            numeric_cols = current_data_with_features.select_dtypes(include=[np.number]).columns
            current_data_with_features[numeric_cols] = current_data_with_features[numeric_cols].apply(pd.to_numeric, errors='coerce')
            
            # Handle potential NaN or Inf values
            current_data_with_features.replace([np.inf, -np.inf], np.nan, inplace=True)
            current_data_with_features.fillna(method='bfill', inplace=True)
            current_data_with_features.fillna(method='ffill', inplace=True)
            
            # Check for NaN or Inf values
            if check_nan_inf(current_data_with_features, day, stock):
                break  # Exit the loop if invalid values are found
            
            # Scale features (excluding 'Close' and 'Date')
            feature_cols = [col for col in current_data_with_features.columns if col not in ['Date', 'Close']]
            X_current_scaled = pd.DataFrame(scaler_X.transform(current_data_with_features[feature_cols]), columns=feature_cols)
            
            # Update current_data with scaled features, retaining 'Close' and 'Date'
            current_data = X_current_scaled.copy()
            current_data['Close'] = scaler_y.transform(current_data_with_features['Close'].values.reshape(-1, 1)).flatten()
            current_data['Date'] = current_data_with_features['Date'].values
            
        except Exception as e:
            print(f" - Error forecasting Day {day} for {stock}: {e}")
            break
    
    if not meta_predictions:
        print(f" - No forecasted results for {stock}. Skipping DataFrame creation.")
        continue
    
    # Create Forecast DataFrame with all model predictions
    forecast_df = pd.DataFrame({
        'Date': forecasted_dates,
        'XGB_Pred': xgb_predictions,
        'RF_Pred': rf_predictions,
        'LSTM_Pred': lstm_predictions,
        'GRU_Pred': gru_predictions,
        'Meta_Pred': meta_predictions
    })
    
    # Display the forecasted results
    print(f"\nForecasted Prices for {stock} - Next {len(forecast_df)} Days:")
    display(forecast_df)
    
    # Calculate and display summary statistics
    forecast_stats = forecast_df[['XGB_Pred', 'RF_Pred', 'LSTM_Pred', 'GRU_Pred', 'Meta_Pred']].describe()
    print(f"\nSummary Statistics for Forecasted Prices of {stock}:")
    display(forecast_stats)
    
    # Save the forecast to CSV
    forecast_save_path = os.path.join(forecast_save_dir, f'future_forecasts_{stock}.csv')
    forecast_df.to_csv(forecast_save_path, index=False)
    print(f" - Future forecasts saved at '{forecast_save_path}'")
